<a href="https://colab.research.google.com/github/suntech123/BigData/blob/master/How_to_create_TFRecords_from_tabular_data_and_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# LOAD LIBRARIES
import numpy as np, pandas as pd, os
import matplotlib.pyplot as plt, cv2
import tensorflow as tf, re, math

In [2]:
# Upload file from Local File System
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving marking.csv to marking.csv
User uploaded file "marking.csv" with length 3225994 bytes


In [50]:
# Upload file from Local File System
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving ISIC_0012549.jpg to ISIC_0012549.jpg
Saving ISIC_0013374.jpg to ISIC_0013374.jpg
Saving ISIC_0013792.jpg to ISIC_0013792.jpg
Saving ISIC_0055803.jpg to ISIC_0055803.jpg
Saving ISIC_2104341.jpg to ISIC_2104341.jpg
Saving ISIC_2110130.jpg to ISIC_2110130.jpg
Saving ISIC_2111943.jpg to ISIC_2111943.jpg
Saving ISIC_2117372.jpg to ISIC_2117372.jpg
Saving ISIC_2118143.jpg to ISIC_2118143.jpg
Saving ISIC_4620906.jpg to ISIC_4620906.jpg
User uploaded file "ISIC_0012549.jpg" with length 128261 bytes
User uploaded file "ISIC_0013374.jpg" with length 76916 bytes
User uploaded file "ISIC_0013792.jpg" with length 94266 bytes
User uploaded file "ISIC_0055803.jpg" with length 122585 bytes
User uploaded file "ISIC_2104341.jpg" with length 76726 bytes
User uploaded file "ISIC_2110130.jpg" with length 53288 bytes
User uploaded file "ISIC_2111943.jpg" with length 69523 bytes
User uploaded file "ISIC_2117372.jpg" with length 74180 bytes
User uploaded file "ISIC_2118143.jpg" with length 70863 bytes


In [5]:
# LOAD TRAIN META DATA
df = pd.read_csv('marking.csv')
df.rename({'image_id':'image_name'},axis=1,inplace=True)

In [59]:
sample_images = ['ISIC_0012549','ISIC_0013374','ISIC_0013792','ISIC_0055803','ISIC_2104341','ISIC_2110130','ISIC_2111943','ISIC_2117372','ISIC_2118143','ISIC_4620906']
train = df[df['image_name'].isin(sample_images)].copy()

In [60]:
train

,patient_id,image_name,target,source,sex,age_approx,anatom_site_general_challenge
6710,IP_4382720,ISIC_2104341,0,ISIC20,female,50.0,torso
6730,IP_4454844,ISIC_2110130,0,ISIC20,female,35.0,lower extremity
6734,IP_2321962,ISIC_2111943,0,ISIC20,female,30.0,torso
6751,IP_7330399,ISIC_2117372,0,ISIC20,female,55.0,upper extremity
6752,IP_2285480,ISIC_2118143,0,ISIC20,female,45.0,torso
15095,IP_6017204,ISIC_4620906,0,ISIC20,female,35.0,torso
47491,BCN_0003752,ISIC_0055803,1,ISIC19,male,75.0,head/neck
58955,NaN,ISIC_0013374,1,SLATMD,NaN,NaN,NaN
59114,NaN,ISIC_0012549,0,SLATMD,NaN,NaN,NaN
59203,NaN,ISIC_0013792,0,SLATMD,NaN,NaN,NaN


In [61]:
# Label Encode Meta Data
### It is more efficient to store this meta data as integers instead of strings. 
### We will impute the Age NaNs to Age mean. Then all other NaNs will be convert to -1 and the other strings 
### will be converted to 0, 1, 2, 3, ... in the order they appear in the printed lists below.

# LABEL ENCODE ALL STRINGS
cats = ['patient_id','sex','anatom_site_general_challenge'] 
for c in cats:
    train[c],mp = train[c].factorize()  # mp is list of unique values
    print(mp)
print('Imputing Age NaN count =',train.age_approx.isnull().sum())
train.age_approx.fillna(train.age_approx.mean(),inplace=True)
train['age_approx'] = train.age_approx.astype('int')

Index(['IP_4382720', 'IP_4454844', 'IP_2321962', 'IP_7330399', 'IP_2285480',
       'IP_6017204', 'BCN_0003752'],
      dtype='object')
Index(['female', 'male'], dtype='object')
Index(['torso', 'lower extremity', 'upper extremity', 'head/neck'], dtype='object')
Imputing Age NaN count = 3


In [62]:
train

,patient_id,image_name,target,source,sex,age_approx,anatom_site_general_challenge
6710,0,ISIC_2104341,0,ISIC20,0,50,0
6730,1,ISIC_2110130,0,ISIC20,0,35,1
6734,2,ISIC_2111943,0,ISIC20,0,30,0
6751,3,ISIC_2117372,0,ISIC20,0,55,2
6752,4,ISIC_2118143,0,ISIC20,0,45,0
15095,5,ISIC_4620906,0,ISIC20,0,35,0
47491,6,ISIC_0055803,1,ISIC19,1,75,3
58955,-1,ISIC_0013374,1,SLATMD,-1,46,-1
59114,-1,ISIC_0012549,0,SLATMD,-1,46,-1
59203,-1,ISIC_0013792,0,SLATMD,-1,46,-1


In [63]:
# LABEL ENCODE TRAIN SOURCE
train.source,mp = train.source.factorize()
print(mp)

Index(['ISIC20', 'ISIC19', 'SLATMD'], dtype='object')


In [64]:
train

,patient_id,image_name,target,source,sex,age_approx,anatom_site_general_challenge
6710,0,ISIC_2104341,0,0,0,50,0
6730,1,ISIC_2110130,0,0,0,35,1
6734,2,ISIC_2111943,0,0,0,30,0
6751,3,ISIC_2117372,0,0,0,55,2
6752,4,ISIC_2118143,0,0,0,45,0
15095,5,ISIC_4620906,0,0,0,35,0
47491,6,ISIC_0055803,1,1,1,75,3
58955,-1,ISIC_0013374,1,2,-1,46,-1
59114,-1,ISIC_0012549,0,2,-1,46,-1
59203,-1,ISIC_0013792,0,2,-1,46,-1


In [65]:
# Write TFRecords
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [66]:
def serialize_example(feature0, feature1, feature2, feature3, feature4, feature5, feature6, feature7):
  feature = {
      'image': _bytes_feature(feature0),
      'image_name': _bytes_feature(feature1),
      'patient_id': _int64_feature(feature2),
      'sex': _int64_feature(feature3),
      'age_approx': _int64_feature(feature4),
      'anatom_site_general_challenge': _int64_feature(feature5),
      'source': _int64_feature(feature6),
      'target': _int64_feature(feature7)
  }
  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [72]:
# PATHS TO IMAGES
PATH = '/content/melanoma_images/'
IMGS = os.listdir(PATH)
##print(IMGS)

In [73]:
SIZE = 3
CT = len(IMGS)//SIZE + int(len(IMGS)%SIZE!=0)
for j in range(CT):
    print(); print('Writing TFRecord %i of %i...'%(j,CT))
    CT2 = min(SIZE,len(IMGS)-j*SIZE)
    with tf.io.TFRecordWriter('train%.2i-%i.tfrec'%(j,CT2)) as writer:
        for k in range(CT2):
            img = cv2.imread(PATH+IMGS[SIZE*j+k])
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # Fix incorrect colors
            img = cv2.imencode('.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, 94))[1].tostring()
            name = IMGS[SIZE*j+k].split('.')[0]
            row = train.loc[train.image_name==name]
            example = serialize_example(
                img, str.encode(name),
                row.patient_id.values[0],
                row.sex.values[0],
                row.age_approx.values[0],                        
                row.anatom_site_general_challenge.values[0],
                row.source.values[0],
                row.target.values[0])
            writer.write(example)
            if k%100==0: print(k,', ',end='')


Writing TFRecord 0 of 4...
0 , 
Writing TFRecord 1 of 4...
0 , 
Writing TFRecord 2 of 4...
0 , 
Writing TFRecord 3 of 4...
0 , 